### RFC

### import modules and configure notebook

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor

pd.set_option('max.rows', None)
pd.set_option('max.columns', None)
%matplotlib inline

### Load variables stored by data_preproccessing notebook

In [2]:
%store -r train_data_formodel
%store -r test_data
%store -r my_data
%store -r uniques
%store -r best_feats

### configurations
* save_plots -> boolean
* scale -> True|False if set to True then features scaled to all have mean value 0 and standard deviation 1

In [3]:
save_plots = False

#the data should be scaled because LOF model uses KNN
scale = True

### counts of instances in all classes before oversampling

In [4]:
train_data_formodel['class'].value_counts()

4     105
15    100
16     61
0      53
11     45
13     36
14     36
2      36
10     30
7      30
6      30
5      27
8      27
1      24
12     21
3      18
9      17
Name: class, dtype: int64

### The class column is stored as the variable y 

In [5]:
y_train = np.array(train_data_formodel['class'])

### The variables identified as best by feature selection are used as features

In [6]:
train_data_feats = train_data_formodel[best_feats]
test_data_feats = test_data[best_feats]
test_data_identifiers = test_data['Analysis']

### features are standardised

In [7]:
if scale:
    my_scaler = StandardScaler()
    X_test_scaled = np.array(my_scaler.fit_transform(test_data_feats))
    X_train = np.array(my_scaler.fit_transform(train_data_feats))
else:
    X_test = np.array(test_data_feats)
    X_train = np.array(train_data_feats)

### the dimensions of the class and features are checked

In [8]:
print(X_train.shape)
print(X_test_scaled.shape)
print(y_train.shape)

(696, 15)
(363, 15)
(696,)


### Local Outlier Factor model is fitted on data that will be used for training the final classifier model
* The model is used for the purpose of novelty detection

In [9]:
lof = LocalOutlierFactor(novelty=True,n_neighbors=25 )
#set novelty to true, this is suitable because we are trying to assess whether the artefcats come from the same distribution of the geological samples, refer to sklearn documentation for more details 

### print details of model

In [10]:
print(lof)

LocalOutlierFactor(algorithm='auto', contamination='legacy', leaf_size=30,
                   metric='minkowski', metric_params=None, n_jobs=None,
                   n_neighbors=25, novelty=True, p=2)


In [11]:
#train model on geological samples
lof.fit(X_train)

C:\Users\tom\Anaconda3\lib\site-packages\sklearn\neighbors\lof.py:236: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)


LocalOutlierFactor(algorithm='auto', contamination='legacy', leaf_size=30,
                   metric='minkowski', metric_params=None, n_jobs=None,
                   n_neighbors=25, novelty=True, p=2)

### Local Outlier Factor model predicts whether the artefact samples are inliers or outliers 

* inlier = 1
* outier = -1

In [12]:
predictions = lof.predict(X_test_scaled, )

In [13]:
pd.Series(predictions).value_counts()

 1    212
-1    151
dtype: int64

In [14]:
nInliers = list(pd.Series(predictions).value_counts())[0]
nOutliers = list(pd.Series(predictions).value_counts())[1]

In [15]:
print('there are {0} inliers and {1} outliers, so the proportion of inliers is {2}'.format(nInliers, nOutliers, (nInliers)/(X_test_scaled.shape[0])))

there are 212 inliers and 151 outliers, so the proportion of inliers is 0.5840220385674931


### column encoding inlier status is added to artefact-sample dataset 

In [16]:
X_test_df = pd.DataFrame(data = np.array(test_data_feats), columns = test_data_feats.columns.values)
X_test_labeled_df = pd.concat([test_data_identifiers.reset_index(drop = True), X_test_df, pd.Series(predictions)], axis =1).rename(columns = {0:'inlierLabel'})

In [17]:
X_test_labeled_df.head()

,Analysis,Zr90,Ba137,Sr88,Ge72,Cr52,S33,U238,Al27,B11,Mg24,Nd146,Sc45,K39,Pr141,Li7,inlierLabel
0,06_DH1_1,1.43,5.95,2.00,1.37,5.18,806.55,0.250083,635.240000,93.21,27.83,0.34,1.08,443.100000,0.07,2.79,1
1,07_DH1_2,1.47,7.51,1.84,1.93,5.43,807.55,0.250083,639.090000,95.19,27.99,0.24,1.09,442.100000,0.06,2.37,-1
2,08_DH1_3,1.16,4.40,1.93,2.14,2.61,837.73,0.250083,670.480000,93.63,26.43,0.08,1.17,444.280000,0.03,2.46,-1
3,09_DH2_1,1.45,12.48,4.29,1.59,2.87,750.62,0.080000,1515.220000,63.80,31.98,0.25,1.19,369.135675,0.06,18.47,1
4,10_DH2_2,1.99,14.40,4.44,0.81,5.92,796.21,0.090000,710.920579,60.96,44.95,0.51,1.22,369.135675,0.12,19.98,1


### dataset is stored

In [18]:
%store X_test_labeled_df

Stored 'X_test_labeled_df' (DataFrame)
